# Voici le site à scrapper
 https://www.basketball-reference.com/leagues/NBA_2022_per_game.html

In [47]:
# Import des bibliothèques nécessaires  
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io import sql
import subprocess
from urllib.parse import quote_plus

## Partie 1 : Prise en main Scraping avec BeautifulSoup

Outils : Python, BeautifulSoup, Request, Outil de développeur de Chrome/Firefox, ...

Pour la prise en main de BeautifulSoup je vous propose un ensemble de cas pratiques simples (voir fichier pdf).

Pour chacun de ces cas, vous devez rechercher l'organisation HTML du site, et notamment les noms et type de balise permettant de repérer les informations utiles. Ecrire un script python qui récupère des informations par scraping

Allé plus loin en explorant des sites plus complexes...

Remarque : Vous pouvez scrapper "n'importe" quel site Internet que vous pouvez consulter, mais la difficulté de cette opération dépend du site. D'autres exemples de sites : http://books.toscrape.com/ ; https://realpython.github.io/fake-jobs/ ; https://www.gov.uk

​

### Cas pratique 1
Voici le site à scrapper

https://www.basketball-reference.com/leagues/NBA_2022_per_game.html

In [4]:
# L'url du site à scrapper
url = 'https://www.basketball-reference.com/leagues/NBA_2022_per_game.html'

# ma requete HTTP avec un "GET" au serveur du site identifié dans l'url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)

# J'affiche l'url ainsi que le retour du serveur
print(url, response.status_code)


https://www.basketball-reference.com/leagues/NBA_2022_per_game.html 200


In [6]:
# Je demande à beautifulSoup de conserver dans une variable "soup" la page web à scrapper (un script html)
soup = BeautifulSoup(response.content,'html')
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202410171" rel="dns-prefetch"/>
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp.osano.com/16CGnCU8UtNhM14sg/12669873-8cf8-41

In [8]:

# URL du site  
url = "https://www.basketball-reference.com/leagues/NBA_2022_per_game.html"  

# Récupération du contenu HTML  
response = requests.get(url)  
soup = BeautifulSoup(response.content, 'html.parser')  

# Recherche du tableau de statistiques  
table = soup.find('table', {'id': 'per_game_stats'})  

# Extraction des en-têtes (modification ici)  
headers = []  
for th in table.find('thead').find_all('th'):  
    # On ne prend que le texte des en-têtes qui ont un attribut 'data-stat'  
    if th.get('data-stat'):  
        headers.append(th.get('data-stat'))  

# Extraction des données des joueurs  
rows = []  
for tr in table.find('tbody').find_all('tr'):  
    # Vérifier si la ligne n'est pas une ligne de séparation  
    if not tr.get('class') or 'thead' not in tr.get('class'):  
        row = []  
        for td in tr.find_all(['td', 'th']):  
            row.append(td.text.strip())  
        if len(row) > 0:  # Certaines lignes peuvent être vides  
            rows.append(row)  

In [10]:
# Création du DataFrame  
df = pd.DataFrame(rows, columns=headers)  

# Nettoyage des données  
df = df.dropna(how='all')  
df = df.drop_duplicates()  

# Export en CSV  
df.to_csv('nba_stats_2022.csv', index=False)  

# Affichage des premières lignes  
df

,ranker,name_display,age,team_name_abbr,pos,games,games_started,mp_per_g,fg_per_g,fga_per_g,...,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,awards
0,1,Joel Embiid,27,PHI,C,68,68,33.8,9.8,19.6,...,2.1,9.6,11.7,4.2,1.1,1.5,3.1,2.7,30.6,"MVP-2,AS,NBA2"
1,2,LeBron James,37,LAL,C,56,56,37.2,11.4,21.8,...,1.1,7.1,8.2,6.2,1.3,1.1,3.5,2.2,30.3,"MVP-10,AS,NBA3"
2,3,Giannis Antetokounmpo,27,MIL,PF,67,67,32.9,10.3,18.6,...,2.0,9.6,11.6,5.8,1.1,1.4,3.3,3.2,29.9,"MVP-3,DPOY-6,AS,NBA1"
3,4,Kevin Durant,33,BRK,PF,55,55,37.2,10.5,20.3,...,0.5,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,"MVP-10,AS,NBA2"
4,5,Luka Dončić,22,DAL,PG,65,65,35.4,9.9,21.6,...,0.9,8.3,9.1,8.7,1.2,0.6,4.5,2.2,28.4,"MVP-5,AS,NBA1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,602,Trayvon Palmer,27,DET,SG,1,0,17.0,0.0,1.0,...,0.0,2.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,
809,603,Emanuel Terry,25,PHO,PF,3,0,6.0,0.0,1.7,...,2.7,2.3,5.0,0.7,0.3,0.0,1.7,1.3,0.0,
810,604,Jon Teske,24,MEM,C,3,0,2.7,0.0,0.3,...,0.0,0.7,0.7,0.3,0.3,0.0,0.0,0.3,0.0,
811,605,M.J. Walker,23,PHO,SG,2,0,4.0,0.0,2.0,...,0.0,0.5,0.5,0.5,1.0,0.0,0.0,0.5,0.0,


In [26]:
import pandas as pd  
import requests  
from bs4 import BeautifulSoup  

# URL du site  
url = "https://www.basketball-reference.com/leagues/NBA_2022_per_game.html"  

# Récupération du contenu HTML  
response = requests.get(url)  
soup = BeautifulSoup(response.content, 'html.parser')  

# Trouver le tableau  
table = soup.find('table', {'id': 'per_game_stats'})  

# Extraction des en-têtes en utilisant les attributs HTML appropriés  
headers = []  
for th in table.find('thead').find_all('th'):  
    # Afficher les attributs de chaque en-tête pour debug  
    print(f"Texte: {th.text.strip()}")  
    print(f"Attributs: {th.attrs}")  
    print("---")  
    
    # Le nom de la colonne est soit dans data-stat, soit dans le texte  
    header_text = th.get('data-stat', th.text.strip())  
    headers.append(header_text)  

print("\nListe complète des en-têtes trouvés:")  
print(headers)

Texte: Rk
Attributs: {'aria-label': 'Rk', 'data-stat': 'ranker', 'scope': 'col', 'class': ['ranker', 'poptip', 'center'], 'data-tip': 'Rank'}
---
Texte: Player
Attributs: {'aria-label': 'Player', 'data-stat': 'name_display', 'scope': 'col', 'class': ['poptip', 'sort_default_asc', 'center']}
---
Texte: Age
Attributs: {'aria-label': 'Age', 'data-stat': 'age', 'scope': 'col', 'class': ['poptip', 'center'], 'data-tip': "Player's age on February 1 of the season"}
---
Texte: Team
Attributs: {'aria-label': 'Team', 'data-stat': 'team_name_abbr', 'scope': 'col', 'class': ['poptip', 'sort_default_asc', 'center']}
---
Texte: Pos
Attributs: {'aria-label': 'Pos', 'data-stat': 'pos', 'scope': 'col', 'class': ['poptip', 'center'], 'data-tip': 'Position'}
---
Texte: G
Attributs: {'aria-label': 'G', 'data-stat': 'games', 'scope': 'col', 'class': ['poptip', 'center'], 'data-tip': 'Games'}
---
Texte: GS
Attributs: {'aria-label': 'GS', 'data-stat': 'games_started', 'scope': 'col', 'class': ['poptip', 'cen


### Cas pratique 2

### Voici le site à scrapper
https://content.codecademy.com/courses/beautifulsoup/cacao/index.html

In [15]:
# L'url du site à scrapper
url = 'https://content.codecademy.com/courses/beautifulsoup/cacao/index.html'

# ma requete HTTP avec un "GET" au serveur du site identifié dans l'url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)

# J'affiche l'url ainsi que le retour du serveur
print(url, response.status_code)

https://content.codecademy.com/courses/beautifulsoup/cacao/index.html 200


In [17]:
# Je demande à beautifulSoup de conserver dans une variable "soup" la page web à scrapper (un script html)
soup = BeautifulSoup(response.content,'html')
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
         #cacaoTable {
         border-collapse: collapse;
         border: 2px black solid;
         font: 12px sans-serif;
         }
         #cacaoTable td {
         border: 1px black solid;
         padding: 5px;
         }
         #infoContainer {
         margin-bottom: 10px;
         display: inline-block;
         margin-right: 10px;
         }
         #chocolate {
         width: 300px;
         }
         #picContainer {
         display: inline;
         }
      </style>
</head>
<body>
<!-- <script src="http://d3js.org/d3.v3.min.js"></script> -->
<div id="banner">
<h1>Cacao Ratings</h1>
</div>
<div id="infoContainer">
<table class="infoTable">
<tr>
<td>Compiled ratings of over 1700 Chocolate bars</td>
</tr>
<tr>
<td>Ratings are from 1-5</td>
</tr>
</table>
</div>
<div id="picContainer">
<img id="chocolate" src="chocolate.jpg"/>
</div>
<table id="cacaoTable">
<tr>
<td class="Company">Company 
        

In [81]:

# URL  
url = "https://content.codecademy.com/courses/beautifulsoup/cacao/index.html"  

# Request the page  
response = requests.get(url)  
soup_choco = BeautifulSoup(response.content, 'html.parser')  

# Find all tables  
tables = soup_choco.find_all("table")  

if tables:  
    print(f"{len(tables)} tableau(x) trouvé(s).")  

    # Iterate over each table and print its content  
    for idx, table in enumerate(tables):  
        print(f"\nTableau {idx + 1}")  
        rows = table.find_all("tr")  

        for row in rows:  
            columns = row.find_all("td")  
            print([col.get_text().strip() for col in columns])  # Print each cell's text  

    # Assuming you know the relevant table  
    table = tables[0]  # Modify index based on investigation  
    rows = table.find_all("tr")  

    # Lists to hold data  
    ratings = [] 
    cocoa_percent = []  

    # Skip the header and process each row  
    for row in rows[1:]:  
        columns = row.find_all("td")  
        # Ensure enough columns exist  
        if len(columns) >= 7:   
            try:  
                # Extract and process the data  
                cocoa = float(columns[4].get_text(strip=True).rstrip('%'))  
                rating = float(columns[5].get_text(strip=True))  
                cocoa_percent.append(cocoa)  
                ratings.append(rating)  
                print(f"Extracted Cocoa Percent: {cocoa}, Rating: {rating}")  
            except (ValueError, IndexError):  
                print(f"Erreur lors de l'extraction des données dans la ligne : {row}")  
else:  
    print("Aucun tableau trouvé.")
    table = soup_choco.find_all("table")[1]
print(table)
rows = table.find_all("tr")

2 tableau(x) trouvé(s).

Tableau 1
['Compiled ratings of over 1700 Chocolate bars']
['Ratings are from 1-5']

Tableau 2
['Company\xa0\n               (Maker-if known)', 'Specific Bean Origin\n               or Bar Name', 'REF', 'Review\n               Date', 'Cocoa\n               Percent', 'Company\n               Location', 'Rating', 'Bean\n               Type', 'Broad Bean\n               Origin']
['A. Morin', 'Agua Grande', '1876', '2016', '63%', 'France', '3.75', '', 'Sao Tome']
['A. Morin', 'Kpime', '1676', '2015', '70%', 'France', '2.75', '', 'Togo']
['A. Morin', 'Atsane', '1676', '2015', '70%', 'France', '3', '', 'Togo']
['A. Morin', 'Akata', '1680', '2015', '70%', 'France', '3.5', '', 'Togo']
['A. Morin', 'Quilla', '1704', '2015', '70%', 'France', '3.5', '', 'Peru']
['A. Morin', 'Carenero', '1315', '2014', '70%', 'France', '2.75', 'Criollo', 'Venezuela']
['A. Morin', 'Cuba', '1315', '2014', '70%', 'France', '3.5', '', 'Cuba']
['A. Morin', 'Sur del Lago', '1315', '2014', '70%',

In [75]:
# Sélectionner le deuxième tableau (index 1)  
table = tables[1]  # Important: on prend le deuxième tableau  
rows = table.find_all("tr")  

# Initialiser les listes vides  
cocoa_percent = []  
ratings = []  

# Extraire les données  
for row in rows[1:]:  
    columns = row.find_all("td")  
    if len(columns) >= 9:   
        try:    
            rating = float(columns[6].get_text()) 
            cocoa = float(columns[4].get_text().strip('%'))
            ratings.append(rating) 
            cocoa_percent.append(cocoa)  
            print(f"Extracted  Rating: {rating}, Cocoa Percent: {cocoa}")  
        except (ValueError, IndexError):  
            print(f"Erreur lors de l'extraction des données dans la ligne : {row}")  

# Créer le DataFrame  
df = pd.DataFrame({  
    'Rating': ratings ,
    'Cocoa Percent': cocoa_percent 
})  

# Exporter le DataFrame en CSV  
df.to_csv('chocolat_ratings.csv', index=False)  
print("\nData exported to 'chocolat_ratings.csv' successfully.")  

# Afficher les premières lignes pour vérification  
print("\nAperçu du DataFrame:")  
df.head() 

# Afficher les dimensions du DataFrame  
print(f"\nNombre total de lignes : {len(df)}")

Extracted  Rating: 3.75, Cocoa Percent: 63.0
Extracted  Rating: 2.75, Cocoa Percent: 70.0
Extracted  Rating: 3.0, Cocoa Percent: 70.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 2.75, Cocoa Percent: 70.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 3.75, Cocoa Percent: 70.0
Extracted  Rating: 4.0, Cocoa Percent: 70.0
Extracted  Rating: 2.75, Cocoa Percent: 70.0
Extracted  Rating: 3.0, Cocoa Percent: 70.0
Extracted  Rating: 3.25, Cocoa Percent: 70.0
Extracted  Rating: 3.75, Cocoa Percent: 70.0
Extracted  Rating: 2.75, Cocoa Percent: 70.0
Extracted  Rating: 3.0, Cocoa Percent: 70.0
Extracted  Rating: 3.25, Cocoa Percent: 70.0
Extracted  Rating: 4.0, Cocoa Percent: 70.0
Extracted  Rating: 3.25, Cocoa Percent: 70.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 4.0, Cocoa Percent: 63.0
Extracted  Rating: 3.5, Cocoa Percent: 70.0
Extracted  Rating: 3.7

In [77]:
df

,Rating,Cocoa Percent
0,3.75,63.0
1,2.75,70.0
2,3.00,70.0
3,3.50,70.0
4,3.50,70.0
...,...,...
1790,3.75,70.0
1791,3.00,65.0
1792,3.50,65.0
1793,3.25,62.0


In [89]:
import pandas as pd  

# Lire le fichier CSV existant  
df = pd.read_csv('chocolat_ratings.csv')  

# Créer les fichiers JSON avec différentes orientations  
# 1. Orient = 'records' (liste de dictionnaires)  
df.to_json('chocolat_ratings_records.json', orient='records')  

# 2. Orient = 'index' (dictionnaire avec index comme clés)  
df.to_json('chocolat_ratings_index.json', orient='index')  

# 3. Orient = 'values' (liste de listes, juste les valeurs)  
df.to_json('chocolat_ratings_values.json', orient='values')  

# Afficher un message de confirmation  
print("Fichiers JSON créés avec succès:")  
print("1. chocolat_ratings_records.json")  
print("2. chocolat_ratings_index.json")  
print("3. chocolat_ratings_values.json")  

# Afficher un aperçu des différents formats  
print("\nAperçu des différents formats:")  
print("\nFormat 'records':")  
print(df.to_json(orient='records')[:200])  
print("\nFormat 'index':")  
print(df.to_json(orient='index')[:200])  
print("\nFormat 'values':")  
print(df.to_json(orient='values')[:200])

Fichiers JSON créés avec succès:
1. chocolat_ratings_records.json
2. chocolat_ratings_index.json
3. chocolat_ratings_values.json

Aperçu des différents formats:

Format 'records':
[{"Rating;Cocoa Percent":"3.75;63.0"},{"Rating;Cocoa Percent":"2.75;70.0"},{"Rating;Cocoa Percent":"3.0;70.0"},{"Rating;Cocoa Percent":"3.5;70.0"},{"Rating;Cocoa Percent":"3.5;70.0"},{"Rating;Cocoa Pe

Format 'index':
{"0":{"Rating;Cocoa Percent":"3.75;63.0"},"1":{"Rating;Cocoa Percent":"2.75;70.0"},"2":{"Rating;Cocoa Percent":"3.0;70.0"},"3":{"Rating;Cocoa Percent":"3.5;70.0"},"4":{"Rating;Cocoa Percent":"3.5;70.0

Format 'values':
[["3.75;63.0"],["2.75;70.0"],["3.0;70.0"],["3.5;70.0"],["3.5;70.0"],["2.75;70.0"],["3.5;70.0"],["3.5;70.0"],["3.75;70.0"],["4.0;70.0"],["2.75;70.0"],["3.0;70.0"],["3.25;70.0"],["3.75;70.0"],["2.75;70.


In [97]:
import pandas as pd  
import json  

# Lire le fichier CSV avec le bon séparateur  
df = pd.read_csv('chocolat_ratings.csv', sep=';')  

# Afficher les noms des colonnes pour vérification  
print("Noms des colonnes dans le DataFrame:")  
print(df.columns.tolist())  

# Nettoyer les noms de colonnes  
df.columns = df.columns.str.strip()  

# Convertir le DataFrame en une liste de dictionnaires simple  
data = []  
for _, row in df.iterrows():  
    chocolate_data = {  
        "Cocoa_Percent": float(row["Cocoa Percent"]),  
        "Rating": float(row["Rating"])  
    }  
    data.append(chocolate_data)  

# Sauvegarder en JSON avec un format lisible  
with open('cocorating_simple.json', 'w', encoding='utf-8') as f:  
    json.dump(data, f, indent=2, ensure_ascii=False)  

print("\nLe fichier cocorating_simple.json a été créé avec succès!")  

# Afficher un aperçu du contenu  
print("\nAperçu du contenu (3 premières entrées):")  
print(json.dumps(data[:3], indent=2))

Noms des colonnes dans le DataFrame:
['Rating', 'Cocoa Percent']

Le fichier cocorating_simple.json a été créé avec succès!

Aperçu du contenu (3 premières entrées):
[
  {
    "Cocoa_Percent": 63.0,
    "Rating": 3.75
  },
  {
    "Cocoa_Percent": 70.0,
    "Rating": 2.75
  },
  {
    "Cocoa_Percent": 70.0,
    "Rating": 3.0
  }
]


### Cas pratique 3
## Voici le site à Scrapper
https://webscraper.io/test-sites/e-commerce/static/computers/laptops

https://webscraper.io/test-sites/e-commerce/static/computers/tablets


In [107]:
import requests  
from bs4 import BeautifulSoup  
import pandas as pd  

# URLs des pages à scraper  
laptop_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops"  
tablet_url = "https://webscraper.io/test-sites/e-commerce/static/computers/tablets"  

def scrape_products(url):  
    # Faire la requête HTTP  
    response = requests.get(url)  
    
    # Vérifier si la requête a réussi  
    if response.status_code == 200:  
        # Parser le contenu HTML  
        soup = BeautifulSoup(response.text, 'html.parser')  
        
        # Listes pour stocker les données  
        names = []  
        descriptions = []  
        prices = []  
        
        # Trouver tous les produits  
        products = soup.find_all('div', class_='caption')  
        
        # Extraire les informations pour chaque produit  
        for product in products:  
            # Extraire le nom  
            name = product.find('a', class_='title').text.strip()  
            names.append(name)  
            
            # Extraire la description  
            description = product.find('p', class_='description').text.strip()  
            descriptions.append(description)  
            
            # Extraire le prix  
            price = product.find('h4', class_='price').text.strip()  
            # Nettoyer le prix (enlever le symbole $ et convertir en float)  
            price = float(price.replace('$', ''))  
            prices.append(price)  
        
        # Créer un DataFrame  
        df = pd.DataFrame({  
            'Nom': names,  
            'Description': descriptions,  
            'Prix': prices  
        })  
        
        return df  
    else:  
        print(f"Erreur lors de la requête. Status code: {response.status_code}")  
        return None  


In [109]:
# Scraper les laptops  
print("Scraping des laptops...")  
laptops_df = scrape_products(laptop_url)  
print("\nDataFrame des laptops :")  
laptops_df  

Scraping des laptops...

DataFrame des laptops :


,Nom,Description,Prix
0,Packard 255 G2,"15.6"", AMD E2-3800 1.3GHz, 4GB, 500GB, Windows...",416.99
1,Aspire E1-510,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",306.99
2,ThinkPad T540p,"15.6"", Core i5-4200M, 4GB, 500GB, Win7 Pro 64bit",1178.99
3,ProBook,"14"", Core i5 2.6GHz, 4GB, 500GB, Win7 Pro 64bit",739.99
4,ThinkPad X240,"12.5"", Core i5-4300U, 8GB, 240GB SSD, Win7 Pro...",1311.99
5,Aspire E1-572G,"15.6"", Core i5-4200U, 8GB, 1TB, Radeon R7 M265...",581.99


In [127]:
# Scraper les tablets de la page 1
print("\nScraping des tablets...")  
tablets_df = scrape_products(tablet_url)  
print("\nDataFrame des tablets :")  
tablets_df


Scraping des tablets...

DataFrame des tablets :


,Nom,Description,Prix
0,Lenovo IdeaTab,"7"" screen, Android",69.99
1,Acer Iconia,"7"" screen, Android, 16GB",96.99
2,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99
3,Amazon Kindle,"6"" screen, wifi",103.99
4,iPad Mini Reti...,"Wi-Fi + Cellular, 32GB, Silver",537.99
5,IdeaTab A3500L,"Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2",88.99


In [ ]:
# Sauvegarder les données dans des fichiers CSV (optionnel)  
#laptops_df.to_csv('laptops.csv', index=False)  
#tablets_df.to_csv('tablets.csv', index=False)

In [129]:
#Pour recupérer les données des  tablets
def get_number_of_pages(url):  
    response = requests.get(url)  
    soup = BeautifulSoup(response.text, 'html.parser')  
    pagination = soup.find('ul', class_='pagination')  
    if pagination:  
        pages = pagination.find_all('li')  
        return int(pages[-2].text)  
    return 1  

def scrape_tablets():  
    base_url = "https://webscraper.io/test-sites/e-commerce/static/computers/tablets"  
    
    all_names = []  
    all_descriptions = []  
    all_prices = []  
    all_ratings = []  
    
    total_pages = get_number_of_pages(base_url)  
    print(f"Nombre total de pages à scraper : {total_pages}")  
    
    for page in range(1, total_pages + 1):  
        page_url = f"{base_url}?page={page}" if page > 1 else base_url  
        print(f"Scraping de la page {page}...")  
        
        response = requests.get(page_url)  
        if response.status_code == 200:  
            soup = BeautifulSoup(response.text, 'html.parser')  
            products = soup.find_all('div', class_='thumbnail')  
            
            for product in products:  
                # Nom  
                name = product.find('a', class_='title').text.strip()  
                all_names.append(name)  
                
                # Description  
                description = product.find('p', class_='description').text.strip()  
                all_descriptions.append(description)  
                
                # Prix  
                price = product.find('h4', class_='price').text.strip()  
                price = float(price.replace('$', ''))  
                all_prices.append(price)  
                
                # Rating  
                rating_element = product.find('div', class_='ratings')  
                if rating_element:  
                    rating = rating_element.find('p', {'data-rating': True})  
                    if rating:  
                        rating_value = float(rating['data-rating'])  
                        all_ratings.append(rating_value)  
                    else:  
                        all_ratings.append(0)  
                else:  
                    all_ratings.append(0)  
        else:  
            print(f"Erreur lors de la requête de la page {page}. Status code: {response.status_code}")  
    
    # Créer le DataFrame  
    df = pd.DataFrame({  
        'Nom': all_names,  
        'Description': all_descriptions,  
        'Prix': all_prices,  
        'Rating': all_ratings  
    })  
    
    return df  

In [131]:

# Exécuter le scraping  
print("Début du scraping des tablets...")  
tablets_df = scrape_tablets()  

# Sauvegarder dans un CSV  
tablets_df.to_csv('tablets_complet.csv', index=False)  
print("\nLes données ont été sauvegardées dans 'tablets_complet.csv'")  

# Afficher les statistiques  
print("\nStatistiques :")  
print(f"Nombre total de tablets : {len(tablets_df)}")  
print("\nDistribution des ratings :")  
print(tablets_df['Rating'].value_counts().sort_index())  
print("\nPrix moyen : ${:.2f}".format(tablets_df['Prix'].mean()))  
print("\nRating moyen : {:.2f}".format(tablets_df['Rating'].mean()))  

# Afficher un aperçu des données  
print("\nAperçu des données :")  
tablets_df  

# Afficher quelques informations supplémentaires  
print("\nInformations sur les prix :")  
print("Prix minimum : ${:.2f}".format(tablets_df['Prix'].min()))  
print("Prix maximum : ${:.2f}".format(tablets_df['Prix'].max()))  
print("Prix médian : ${:.2f}".format(tablets_df['Prix'].median()))  

print("\nNombre de produits par rating :")  
tablets_df['Rating'].value_counts().sort_index()

Début du scraping des tablets...
Nombre total de pages à scraper : 4
Scraping de la page 1...
Scraping de la page 2...
Scraping de la page 3...
Scraping de la page 4...

Les données ont été sauvegardées dans 'tablets_complet.csv'

Statistiques :
Nombre total de tablets : 21

Distribution des ratings :
Rating
1.0    2
2.0    8
3.0    6
4.0    5
Name: count, dtype: int64

Prix moyen : $232.04

Rating moyen : 2.67

Aperçu des données :

Informations sur les prix :
Prix minimum : $69.99
Prix maximum : $603.99
Prix médian : $130.99

Nombre de produits par rating :


Rating
1.0    2
2.0    8
3.0    6
4.0    5
Name: count, dtype: int64

In [133]:
tablets_df

,Nom,Description,Prix,Rating
0,Lenovo IdeaTab,"7"" screen, Android",69.99,3.0
1,Acer Iconia,"7"" screen, Android, 16GB",96.99,1.0
2,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99,4.0
3,Amazon Kindle,"6"" screen, wifi",103.99,4.0
4,iPad Mini Reti...,"Wi-Fi + Cellular, 32GB, Silver",537.99,2.0
5,IdeaTab A3500L,"Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2",88.99,4.0
6,Galaxy Tab,"16GB, White",251.99,3.0
7,IdeaTab A3500-...,"Blue, 7"" IPS, Quad-Core 1.3GHz, 8GB, 3G, Andro...",148.99,2.0
8,Galaxy Tab 3,"7"", 8GB, Wi-Fi, Android 4.2, White",97.99,2.0
9,Galaxy Note,"12.2"", 32GB, WiFi, Android 4.4, White",489.99,3.0


#### Récupérer les informations (nom, description, prix, rating) des laptops de toutes les
#### pages en utilisant BeautifulSoup.
#### Insérer les données récupérées dans un dataframe.

In [121]:
def get_number_of_pages(url):  
    response = requests.get(url)  
    soup = BeautifulSoup(response.text, 'html.parser')  
    pagination = soup.find('ul', class_='pagination')  
    if pagination:  
        pages = pagination.find_all('li')  
        return int(pages[-2].text)  
    return 1  


In [123]:
def scrape_laptops_all_pages():  
    base_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops"  
    
    all_names = []  
    all_descriptions = []  
    all_prices = []  
    all_ratings = []  
    
    total_pages = get_number_of_pages(base_url)  
    print(f"Nombre total de pages à scraper : {total_pages}")  
    
    for page in range(1, total_pages + 1):  
        page_url = f"{base_url}?page={page}" if page > 1 else base_url  
        print(f"Scraping de la page {page}...")  
        
        response = requests.get(page_url)  
        if response.status_code == 200:  
            soup = BeautifulSoup(response.text, 'html.parser')  
            products = soup.find_all('div', class_='thumbnail')  
            
            for product in products:  
                # Nom  
                name = product.find('a', class_='title').text.strip()  
                all_names.append(name)  
                
                # Description  
                description = product.find('p', class_='description').text.strip()  
                all_descriptions.append(description)  
                
                # Prix  
                price = product.find('h4', class_='price').text.strip()  
                price = float(price.replace('$', ''))  
                all_prices.append(price)  
                
                # Rating   
                rating_element = product.find('div', class_='ratings')  
                if rating_element:  
                    # Chercher l'élément data-rating  
                    rating = rating_element.find('p', {'data-rating': True})  
                    if rating:  
                        rating_value = float(rating['data-rating'])  
                        all_ratings.append(rating_value)  
                    else:  
                        all_ratings.append(0)  
                else:  
                    all_ratings.append(0)  
        else:  
            print(f"Erreur lors de la requête de la page {page}. Status code: {response.status_code}")  
    
    df = pd.DataFrame({  
        'Nom': all_names,  
        'Description': all_descriptions,  
        'Prix': all_prices,  
        'Rating': all_ratings  
    })  
    
    return df  



In [125]:
# Exécution du  scraping  
print("Début du scraping...")  
laptops_df = scrape_laptops_all_pages()  

# Sauvegarde des données  
laptops_df.to_csv('laptops_complet.csv', index=False)  
print("\nLes données ont été sauvegardées dans 'laptops_complet.csv'")  

# Affichage des statistiques  
print("\nStatistiques :")  
print(f"Nombre total de laptops : {len(laptops_df)}")  
print("\nDistribution des ratings :")  
print(laptops_df['Rating'].value_counts().sort_index())  
print("\nPrix moyen : ${:.2f}".format(laptops_df['Prix'].mean()))  
print("\nRating moyen : {:.2f}".format(laptops_df['Rating'].mean()))  

# Affichage des résultats  
print("\nAperçu du DataFrame final :")  
laptops_df

Début du scraping...
Nombre total de pages à scraper : 20
Scraping de la page 1...
Scraping de la page 2...
Scraping de la page 3...
Scraping de la page 4...
Scraping de la page 5...
Scraping de la page 6...
Scraping de la page 7...
Scraping de la page 8...
Scraping de la page 9...
Scraping de la page 10...
Scraping de la page 11...
Scraping de la page 12...
Scraping de la page 13...
Scraping de la page 14...
Scraping de la page 15...
Scraping de la page 16...
Scraping de la page 17...
Scraping de la page 18...
Scraping de la page 19...
Scraping de la page 20...

Les données ont été sauvegardées dans 'laptops_complet.csv'

Statistiques :
Nombre total de laptops : 117

Distribution des ratings :
Rating
1.0    37
2.0    24
3.0    35
4.0    21
Name: count, dtype: int64

Prix moyen : $909.39

Rating moyen : 2.34

Aperçu du DataFrame final :


,Nom,Description,Prix,Rating
0,Packard 255 G2,"15.6"", AMD E2-3800 1.3GHz, 4GB, 500GB, Windows...",416.99,2.0
1,Aspire E1-510,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",306.99,3.0
2,ThinkPad T540p,"15.6"", Core i5-4200M, 4GB, 500GB, Win7 Pro 64bit",1178.99,1.0
3,ProBook,"14"", Core i5 2.6GHz, 4GB, 500GB, Win7 Pro 64bit",739.99,4.0
4,ThinkPad X240,"12.5"", Core i5-4300U, 8GB, 240GB SSD, Win7 Pro...",1311.99,3.0
...,...,...,...,...
112,Lenovo ThinkPa...,"Lenovo ThinkPad T470, 14"" FHD IPS, Core i5-720...",1349.23,1.0
113,Lenovo ThinkPa...,"Lenovo ThinkPad Yoga 370 Black, 13.3"" FHD IPS ...",1362.24,2.0
114,Toshiba Porteg...,"Toshiba Portege X20W-D-10V Black/Blue, 12.5"" F...",1366.32,1.0
115,Asus ASUSPRO B...,"Asus ASUSPRO B9440UA-GV0279R Gray, 14"" FHD, Co...",1381.13,1.0


### Code pour les Laptops

In [137]:


# Charger le fichier CSV pour les laptops  
laptops_path = r'C:\\Users\\chris\\Desktop\\Brief_Extract_Data\\laptops_complet.csv'  
laptops_df = pd.read_csv(laptops_path)  

# Fonction pour extraire les données demandées  
def extract_top_bottom(df, rating_col='Rating', price_col='Prix', n=3):  
    top_rated = df.nlargest(n, rating_col)  
    bottom_rated = df.nsmallest(n, rating_col)  
    most_expensive = df.nlargest(n, price_col)  
    least_expensive = df.nsmallest(n, price_col)  
    
    return top_rated, bottom_rated, most_expensive, least_expensive  

# Extraction des données pour les laptops  
laptops_top3, laptops_bottom3, laptops_most_expensive, laptops_least_expensive = extract_top_bottom(laptops_df)  

# Afficher les résultats  
print("Les 3 laptops les mieux notés :")  
display(laptops_top3)  

print("\nLes 3 laptops les moins bien notés :")  
display(laptops_bottom3)  

print("\nLes 3 laptops les plus chers :")  
display(laptops_most_expensive)  

print("\nLes 3 laptops les moins chers :")  
display(laptops_least_expensive)

Les 3 laptops les mieux notés :


,Nom,Description,Prix,Rating
3,ProBook,"14"", Core i5 2.6GHz, 4GB, 500GB, Win7 Pro 64bit",739.99,4.0
15,Dell Vostro 15,"Dell Vostro 15 (3568) Black, 15.6"" FHD, Core i...",488.78,4.0
26,Asus ROG Strix...,"Asus ROG Strix GL702ZC-GC154T, 17.3"" FHD, Ryze...",1769.00,4.0



Les 3 laptops les moins bien notés :


,Nom,Description,Prix,Rating
2,ThinkPad T540p,"15.6"", Core i5-4200M, 4GB, 500GB, Win7 Pro 64bit",1178.99,1.0
5,Aspire E1-572G,"15.6"", Core i5-4200U, 8GB, 1TB, Radeon R7 M265...",581.99,1.0
7,Pavilion,"15.6"", Core i5-4200U, 6GB, 750GB, Windows 8.1",609.99,1.0



Les 3 laptops les plus chers :


,Nom,Description,Prix,Rating
28,Asus ROG Strix...,"Asus ROG Strix SCAR Edition GL503VM-ED115T, 15...",1799.0,3.0
26,Asus ROG Strix...,"Asus ROG Strix GL702ZC-GC154T, 17.3"" FHD, Ryze...",1769.0,4.0
27,Asus ROG Strix...,"Asus ROG Strix GL702ZC-GC209T, 17.3"" FHD IPS, ...",1769.0,1.0



Les 3 laptops les moins chers :


,Nom,Description,Prix,Rating
29,Asus VivoBook...,"Asus VivoBook X441NA-GA190 Chocolate Black, 14...",295.99,3.0
30,Prestigio Smar...,"Prestigio SmartBook 133S Dark Grey, 13.3"" FHD ...",299.00,2.0
31,Prestigio Smar...,"Prestigio SmartBook 133S Gold, 13.3"" FHD IPS, ...",299.00,4.0


### Code pour les Tablettes 

In [139]:


# Charger le fichier CSV pour les tablets  
tablets_path = r'C:\\Users\\chris\\Desktop\\Brief_Extract_Data\\tablets_complet.csv'  
tablets_df = pd.read_csv(tablets_path)  

# Fonction pour extraire les données demandées  
def extract_top_bottom(df, rating_col='Rating', price_col='Prix', n=3):  
    top_rated = df.nlargest(n, rating_col)  
    bottom_rated = df.nsmallest(n, rating_col)  
    most_expensive = df.nlargest(n, price_col)  
    least_expensive = df.nsmallest(n, price_col)  
    
    return top_rated, bottom_rated, most_expensive, least_expensive  

# Extraction des données pour les tablets  
tablets_top3, tablets_bottom3, tablets_most_expensive, tablets_least_expensive = extract_top_bottom(tablets_df)  

# Afficher les résultats  
print("Les 3 tablets les mieux notés :")  
display(tablets_top3)  

print("\nLes 3 tablets les moins bien notés :")  
display(tablets_bottom3)  

print("\nLes 3 tablets les plus chers :")  
display(tablets_most_expensive)  

print("\nLes 3 tablets les moins chers :")  
display(tablets_least_expensive)

Les 3 tablets les mieux notés :


,Nom,Description,Prix,Rating
2,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99,4.0
3,Amazon Kindle,"6"" screen, wifi",103.99,4.0
5,IdeaTab A3500L,"Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2",88.99,4.0



Les 3 tablets les moins bien notés :


,Nom,Description,Prix,Rating
1,Acer Iconia,"7"" screen, Android, 16GB",96.99,1.0
10,MeMO Pad 7,"White, 7"", Atom 1.2GHz, 8GB, Android 4.4",130.99,1.0
4,iPad Mini Reti...,"Wi-Fi + Cellular, 32GB, Silver",537.99,2.0



Les 3 tablets les plus chers :


,Nom,Description,Prix,Rating
20,Apple iPad Air,"Wi-Fi, 64GB, Silver",603.99,3.0
14,Galaxy Note 10...,"10.1"", 32GB, Black",587.99,2.0
4,iPad Mini Reti...,"Wi-Fi + Cellular, 32GB, Silver",537.99,2.0



Les 3 tablets les moins chers :


,Nom,Description,Prix,Rating
0,Lenovo IdeaTab,"7"" screen, Android",69.99,3.0
5,IdeaTab A3500L,"Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2",88.99,4.0
1,Acer Iconia,"7"" screen, Android, 16GB",96.99,1.0


In [ ]:
#pip install selenium webdriver-manager pandas tqdm

### Partie 2 : Scraping Dynamique avec Selenium

Outils : Python, Selenium, Outil de développeur de Chrome/Firefox, ...

Utiliser Selenium pour extraire des données sur un site dynamique

Exemple site : https://www.adamchoi.co.uk/overs/detailed.

Ce site fournit des statistiques détaillées sur les matchs de football.

​

In [2]:
from selenium import webdriver  
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from webdriver_manager.chrome import ChromeDriverManager  # Ajout de cet import  
import pandas as pd  
import time  

def setup_driver():  
    # Configuration du Chrome Driver  
    options = webdriver.ChromeOptions()  
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")  
    
    # Installation automatique du ChromeDriver  
    service = Service(ChromeDriverManager().install())  # Suppression de la version spécifique  
    driver = webdriver.Chrome(service=service, options=options)  
    
    return driver  

In [12]:
#je crée une fonction pour faciliter l'opération de scraping

def scrape_football_stats():  
    driver = setup_driver()  
    
    try:  
        # j'accède au site  
        print("Accès au site...")  
        driver.get("https://www.adamchoi.co.uk/overs/detailed")  
        
        # j'attend que la page soit chargée  
        WebDriverWait(driver, 10).until(  
            EC.presence_of_element_located((By.XPATH, "//div[@class='panel-body']"))  
        )  
        
        # je clique sur le bouton "All matches" s'il existe  
        try:  
            all_matches_button = WebDriverWait(driver, 10).until(  
                EC.element_to_be_clickable((By.XPATH, "//label[contains(text(),'All matches')]"))  
            )  
            all_matches_button.click()  
            print("Bouton 'All matches' cliqué")  
            time.sleep(2)  # ici on attend le chargement des données  
        except:  
            print("Bouton 'All matches' non trouvé ou non cliquable")  
        
        # ici commence l'extraction des données  
        print("Extraction des données...")  
        matches = driver.find_elements(By.XPATH, "//tr")  
        
        # je prépare des listes vides pour stocker les données  
        dates = []  
        homes = []  
        scores = []  
        aways = []  
        
        # je parcoure chaque match  
        for match in matches[1:]:  # on  ignore l'en-tête  
            columns = match.find_elements(By.TAG_NAME, "td")  
            if len(columns) >= 4:  # Vérifier qu'il y a assez de colonnes  
                dates.append(columns[0].text)  
                homes.append(columns[1].text)  
                scores.append(columns[2].text)  
                aways.append(columns[3].text)  
        
        # je crée un DataFrame  
        df = pd.DataFrame({  
            'Date': dates,  
            'Équipe domicile': homes,  
            'Score': scores,  
            'Équipe extérieure': aways  
        })  
 #  je sauvegarde le résultat dans un fichier csv en CSV  
       # df.to_csv('football_matches.csv', index=False)  j'ai mis cette ligne en commentaire car elle a déjé été exécutée 
        #et le fichier est déjà dans un dossier en local
        print(f"Données sauvegardées dans 'football_matches.csv'. {len(df)} matchs extraits.")  
        
        # j'affiche un aperçu  
        print("\nAperçu des données :")  
        display(df.head())  
        
        return df  
        
    except Exception as e:  
        print(f"Une erreur est survenue : {str(e)}")  
        return None  
        
    finally:  
        # je ferme le navigateur  
        driver.quit()  

In [14]:
# Fonction principale  
if __name__ == "__main__":  
    print("Début du scraping...")  
    df = scrape_football_stats()  
    
    if df is not None:  
        # j'affiche quelques statistiques  
        print("\nStatistiques :")  
        print(f"Nombre total de matchs : {len(df)}")  
        
        # j'analyse les scores  
        df['Buts totaux'] = df['Score'].apply(lambda x: sum(map(int, x.split('-'))))  
        print("\nMoyenne de buts par match :", df['Buts totaux'].mean())  
        
        # Un exemple avec le  top 5 des matchs avec le plus de buts  
        print("\nTop 5 des matchs avec le plus de buts :")  
        display(df.nlargest(5, 'Buts totaux')[['Date', 'Équipe domicile', 'Score', 'Équipe extérieure', 'Buts totaux']])

Début du scraping...
Accès au site...
Bouton 'All matches' cliqué
Extraction des données...
Données sauvegardées dans 'football_matches.csv'. 159 matchs extraits.

Aperçu des données :


,Date,Équipe domicile,Score,Équipe extérieure
0,24-08-2024,Aston Villa,0 - 2,Arsenal
1,31-08-2024,Arsenal,1 - 1,Brighton
2,15-09-2024,Tottenham,0 - 1,Arsenal
3,22-09-2024,Man City,2 - 2,Arsenal
4,28-09-2024,Arsenal,4 - 2,Leicester



Statistiques :
Nombre total de matchs : 159

Moyenne de buts par match : 2.880503144654088

Top 5 des matchs avec le plus de buts :


,Date,Équipe domicile,Score,Équipe extérieure,Buts totaux
29,05-10-2024,Brentford,5 - 3,Wolves,8
40,25-08-2024,Wolves,2 - 6,Chelsea,8
152,25-08-2024,Wolves,2 - 6,Chelsea,8
157,05-10-2024,Brentford,5 - 3,Wolves,8
4,28-09-2024,Arsenal,4 - 2,Leicester,6


In [2]:
# pip install requests pandas python-dotenv tqdm